In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

In [3]:
### Забираем данные из таблиц

user_data = pd.read_sql(
    """SELECT * FROM public.user_data""",
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

post_text_df = pd.read_sql(
    """SELECT * FROM public.post_text_df""",
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

In [4]:
### Ранее скачанная большая таблица

feed_data = pd.read_csv('feed_data.csv')

In [14]:
### Для дальнейшей обработки разобъем ее на 10 равных частей

part = feed_data.shape[0] // 10

feed_data_1 = feed_data[:part]
feed_data_2 = feed_data[part:part*2]
feed_data_3 = feed_data[part*2:part*3]
feed_data_4 = feed_data[part*3:part*4]
feed_data_5 = feed_data[part*4:part*5]
feed_data_6 = feed_data[part*5:part*6]
feed_data_7 = feed_data[part*6:part*7]
feed_data_8 = feed_data[part*7:part*8]
feed_data_9 = feed_data[part*8:part*9]
feed_data_10 = feed_data[part*9:]

In [108]:
### Пустая таблица, куда будет складываться урезанное количество записей по каждому юзеру

feed_data_new = pd.DataFrame(columns=feed_data.columns)

Что мы делаем: из первой части таблицы забираем 5% данных о кажом найденном в ней юзере и сохраняем в `feed_data_new`. Пользователи, отсутствующие в данной таблице складываются в список `missed_users`. Далее в цикле проходимся по всем остальным частям таблицы и собираем информацию по пользователям в `missed_users`.

In [89]:
def user_search(feed_data, users):
    
    feed_data_reduced = pd.DataFrame(columns=feed_data.columns)
    missed_users = []

    for user in users:
            
            actions = feed_data[feed_data['user_id']==user]
            actions_count = actions.shape[0]

            if actions_count:
                actions = actions[:int(np.ceil(actions_count*0.05))]
                feed_data_reduced = feed_data_reduced.append(actions)
            else:
                missed_users.append(user)
                 
    return feed_data_reduced, missed_users

In [96]:
feed_data_list = [feed_data_2, feed_data_3, feed_data_4,
                  feed_data_5, feed_data_6, feed_data_7,
                  feed_data_8, feed_data_9, feed_data_10]

In [109]:
feed_data_reduced, missed_users = user_search(feed_data_1, user_data['user_id'].unique())
feed_data_new = feed_data_new.append(feed_data_reduced)

In [110]:
i = 0

for feed_data in feed_data_list:
    feed_data_reduced, missed_users = user_search(feed_data, missed_users)
    feed_data_new = feed_data_new.append(feed_data_reduced)
    i += 1
    print(f'Status i: {i}')

Status i: 1
Status i: 2
Status i: 3
Status i: 4
Status i: 5
Status i: 6
Status i: 7
Status i: 8
Status i: 9


In [111]:
feed_data_new.to_csv('feed_data_new.csv', sep=',', index=False)